In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Momenteel vooral nog spielerei, dus weinig consistentie in naamgeving 

In [25]:
neutral = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule
ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with a depleted electron

In [26]:
def C_RHF(molecule):
    
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [27]:
C_neutral = C_RHF(neutral)
C_neutral.matrix()

N: 10
N_P: 5
K: 7


array([[-9.94434582e-01, -2.39158849e-01, -3.32960722e-16,
        -9.36832455e-02,  5.87023442e-31,  1.11639928e-01,
         2.11636262e-16],
       [-2.40970420e-02,  8.85735585e-01,  1.66639772e-15,
         4.79585875e-01, -3.05981991e-30, -6.69579082e-01,
        -1.48550526e-15],
       [-3.24994537e-20, -1.21468321e-16, -6.07284831e-01,
         9.08542044e-16, -8.75583522e-17, -3.24740232e-15,
         9.19234262e-01],
       [-3.16154905e-03,  8.58962042e-02, -2.16029801e-16,
        -7.47431397e-01,  4.52593533e-30, -7.38488587e-01,
        -1.88737912e-15],
       [-1.04013067e-33, -5.61902229e-32, -1.32383143e-16,
         6.51408545e-30,  9.99999990e-01, -3.64134276e-31,
         2.91111017e-17],
       [ 4.59374352e-03,  1.44039556e-01, -4.52997743e-01,
        -3.29471163e-01, -1.66566401e-17,  7.09849504e-01,
        -7.32460670e-01],
       [ 4.59374352e-03,  1.44039556e-01,  4.52997743e-01,
        -3.29471163e-01,  1.66566401e-17,  7.09849504e-01,
         7.3246067

In [28]:
C_ion = C_RHF(ion)
C_ion.matrix()

N: 9
N_P: 4
K: 7


ValueError: RHFSCFEnvironment::RHFSCFEnvironment(const size_t, const RSQHamiltonian<Scalar>&, const SquareMatrix<Scalar>&, const RTransformation<Scalar>&): You have given an odd number of electrons.

In [4]:

N = molecule.numberOfElectrons()
N_P = molecule.numberOfElectronPairs()

spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
K = spin_orbital_basis.numberOfSpatialOrbitals()

hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
S = spin_orbital_basis.quantizeOverlapOperator().parameters()

environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N - 2, hamiltonian, S)
solver = gqcpy.RHFSCFSolver.Plain()
objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()

In [5]:
C_ion = rhf_parameters.expansion()

In [6]:
U = C_ion.inverse()

In [7]:
C_ion.matrix()

array([[-9.94855363e-01, -2.48187030e-01,  1.02446356e-16,
        -7.18584905e-02,  2.01837457e-31, -1.04112821e-01,
         2.09554594e-15],
       [-2.23288578e-02,  9.52560095e-01, -4.76845990e-16,
         3.32694739e-01, -1.14178302e-30,  6.67002137e-01,
        -1.34838170e-14],
       [ 2.71873029e-18, -3.25459809e-17,  8.34864567e-01,
         5.16134121e-16, -5.11912549e-16,  1.49602551e-14,
         7.18879440e-01],
       [-2.79783062e-03,  1.75542120e-01,  3.39413151e-16,
        -9.01766597e-01, -6.03971625e-31,  5.17108673e-01,
        -1.03805852e-14],
       [ 1.44756319e-33, -1.32324229e-31,  4.24514457e-16,
         2.58951114e-30,  9.99999990e-01,  1.35897759e-30,
         2.77780523e-17],
       [ 4.22933465e-03,  6.46948222e-02,  2.36177513e-01,
        -1.40486720e-01,  2.45948937e-16, -7.80555082e-01,
        -8.28206357e-01],
       [ 4.22933465e-03,  6.46948222e-02, -2.36177513e-01,
        -1.40486720e-01, -2.45948937e-16, -7.80555082e-01,
         8.2820635

In [8]:
U = C_ion.inverse()

In [9]:
U = U.transformed(C_neutral)

False

# Dyson orbital spielerei

In [29]:
K = 7
N_P = 2

onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_P, N_P - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

In [30]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_I)
Psi_I.coefficients()

array([ 0.06749438, -0.05083938,  0.09004026,  0.0392738 , -0.13551725,
       -0.13668574, -0.10333404,  0.07960228, -0.13063179,  0.00575358,
        0.1279138 , -0.08330406, -0.08805895, -0.12250621,  0.05952056,
        0.06153853, -0.00160033, -0.07100508,  0.06306163, -0.03625457,
       -0.08922688, -0.07969986, -0.01904737, -0.00909138,  0.10765672,
        0.14224723, -0.01653679, -0.13882504,  0.00708548, -0.13960129,
        0.07546181,  0.02729908,  0.09624197, -0.05079629, -0.07985769,
        0.01088774, -0.02068011, -0.02873894,  0.14525362,  0.13916863,
        0.1044803 ,  0.10954251, -0.05615358, -0.10480192,  0.14831588,
        0.04158678, -0.00951575,  0.12873793,  0.05686232,  0.09401118,
        0.01237995,  0.02170095, -0.10157517,  0.13271277,  0.09577778,
       -0.10270848, -0.13678877, -0.04661311, -0.1162389 , -0.12025714,
        0.00559562, -0.14235353,  0.01832604,  0.10637168,  0.00503969,
        0.05201942,  0.07069313,  0.05915374, -0.14223594, -0.09

In [31]:
Psi_J = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_J)
Psi_J.coefficients()

array([ 5.40541761e-02,  4.61361035e-03, -4.53956273e-02, -5.99679920e-02,
        6.72394429e-02, -4.18356819e-02,  2.54999112e-02, -8.33395811e-02,
        7.33680611e-02, -3.78081029e-02,  6.03803403e-02, -3.72063436e-02,
        7.61441712e-02,  1.14779875e-03, -6.34575934e-02, -1.40571940e-02,
        3.40759530e-02, -1.06885358e-02, -6.60175964e-02,  4.47972447e-02,
       -6.73426571e-02, -9.88506894e-03, -2.61577919e-02, -3.41541187e-02,
        4.33576630e-02, -2.71788509e-02, -2.84833859e-02, -7.47915219e-02,
        1.23166920e-02, -2.47636082e-03, -5.00723133e-02, -6.48839648e-02,
        7.98327921e-02,  6.13638140e-02, -1.37320347e-03, -2.23174629e-02,
        5.13167815e-02, -5.68511301e-02, -5.72927525e-02, -7.44352741e-02,
       -5.90310252e-02, -5.36853369e-03, -2.00936346e-02,  6.45775306e-02,
       -1.27683342e-02, -1.47686785e-02,  2.54628957e-02, -3.22585019e-02,
       -7.40280658e-02,  5.29163229e-02,  6.69256946e-02,  7.06005360e-02,
        6.75250195e-02,  

In [32]:
# Andere namen verzinnen voor de componenten later.
DM_I = Psi_I.calculateSpinResolved1DM()
DM_I_OD = DM_I.orbitalDensity()
DM_I_alpha = DM_I.alpha()
DM_I_beta = DM_I.beta()

DM_J = Psi_J.calculateSpinResolved1DM()
DM_J_OD = DM_J.orbitalDensity()
DM_J_alpha = DM_J.alpha()
DM_J_beta = DM_J.beta()

In [34]:
DM_I.alpha()

array([[ 0.32152512,  0.06739943, -0.06226938, -0.04891287, -0.03118077,
         0.0481638 , -0.01613857],
       [ 0.06739943,  0.32114749, -0.03233346,  0.00892428, -0.04778198,
         0.04494543,  0.01860014],
       [-0.06226938, -0.03233346,  0.32887433,  0.03767751,  0.04222238,
         0.01823684,  0.02450692],
       [-0.04891287,  0.00892428,  0.03767751,  0.26084035, -0.00033152,
         0.0561256 ,  0.04503797],
       [-0.03118077, -0.04778198,  0.04222238, -0.00033152,  0.27941636,
        -0.05950247, -0.00486465],
       [ 0.0481638 ,  0.04494543,  0.01823684,  0.0561256 , -0.05950247,
         0.28860091,  0.05623424],
       [-0.01613857,  0.01860014,  0.02450692,  0.04503797, -0.00486465,
         0.05623424,  0.19959543]])